# Example 4 

## Anomaly detection in fiber optic polarization
In this example, we showcase the capabilities of Learning Entropy, implemented within our AISLEX framework, to detect anomalies in real-world data measured by the CzechLight Polarilog [1]. The original data, previously presented and analyzed in [2], were collected at a sample rate of 20 kHz with 16-bit A/D resolution across four channels.

The fiber optic cable under study spans 21 km, with 10 km running alongside a railway, 5 km within a city, and 6 km in a rural area. This setup is particularly relevant as we utilize AISLEX to detect train movements based on polarization changes. The dataset includes labeled instances of train movement between two stations connected by the fiber optic cable, with the labeling provided by an automated camera-based train detection system.

In this Jupyter notebook, AISLEX has been fine-tuned to detect train movement events such as acceleration and braking, which generate significant electromagnetic fields and noise. Given that part of the cable runs through an urban area, occasional false positives due to external, uncontrollable sources are expected.

For this demonstration, we provide a 19-minute segment of data resampled to 150 Hz for all channels.

[1] R. Vohnout, M. Šlapák, J. Jedlinský, and J. Vojtěch, “CzechLight Polarilog - Rapid Polarization Rotation Monitoring Appliance,” in 2020 22nd International Conference on Transparent Optical Networks (ICTON), Jul. 2020, pp. 1–4. doi: 10.1109/ICTON51198.2020.9203142.

[2] M. Šlapák, J. Vojtěch, O. Havliš, and R. Slavík, “Monitoring of Fibre Optic Links With a Machine Learning-Assisted Low-Cost Polarimeter,” IEEE Access, vol. 8, pp. 183965–183971, 2020, doi: 10.1109/ACCESS.2020.3009524.


In [ ]:
# Import required libs
import math
import pickle
import sys
import time

import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import spectrogram
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler


# Get access to root folder of the notebook
sys.path.append("..")

# Pythonic AISLE
from src.aisle import aisle as py_aisle
from src.aisle import aisle_window as py_aisle_window

# JAX accelerated AISLE
from src.aisle_jax import aisle as jax_aisle
from src.aisle_jax import aisle_window as jax_aisle_window

%matplotlib widget

In [ ]:
# Load the data from the uploaded file
file_path = r'./example_4_data.pkl'

with open(file_path, 'rb') as file:
    data = pickle.load(file)

# Extract the data for each channel
ch1 = data['ch1']
ch2 = data['ch2']
ch3 = data['ch3']
ch4 = data['ch4']

# Perform PCA on the combined data
combined_data = np.vstack([ch1, ch2, ch3, ch4]).T
pca = PCA(n_components=1)
combined_signal = pca.fit_transform(combined_data).flatten()

# Define the sampling frequency and spectogram settings
fs = 150
window = 'hann'
nfft = 256

# Function to plot the spectrogram
def plot_spectrogram(data, fs, ax, title, window, nfft):
    f, t, Sxx = spectrogram(data, fs, window=window, nfft=nfft)
    cax = ax.pcolormesh(t, f, 10 * np.log10(Sxx), shading='gouraud')
    ax.set_ylabel('Frequency [Hz]')
    ax.set_xlabel('Time [sec]')
    ax.set_title(title)
    return cax

fig = plt.figure(figsize=(10, 5))
ax1 = plt.subplot2grid((2, 3), (0, 0))
ax2 = plt.subplot2grid((2, 3), (0, 1))
ax3 = plt.subplot2grid((2, 3), (1, 0))
ax4 = plt.subplot2grid((2, 3), (1, 1))
ax_combined = plt.subplot2grid((2, 3), (0, 2), rowspan=2)

# Plotting
cax1 = plot_spectrogram(ch1, fs, ax1, 'Channel 1', window, nfft)
ax1.xaxis.set_visible(False)
cb1 = fig.colorbar(cax1, ax=ax1)
cb1.set_label('Intensity [dB]')

cax2 = plot_spectrogram(ch2, fs, ax2, 'Channel 2', window, nfft)
ax2.xaxis.set_visible(False)
cb2 = fig.colorbar(cax2, ax=ax2)
cb2.set_label('Intensity [dB]')

cax3 = plot_spectrogram(ch3, fs, ax3, 'Channel 3', window, nfft)
cb3 = fig.colorbar(cax3, ax=ax3)
cb3.set_label('Intensity [dB]')

cax4 = plot_spectrogram(ch4, fs, ax4, 'Channel 4', window, nfft)
cb4 = fig.colorbar(cax4, ax=ax4)
cb4.set_label('Intensity [dB]')

cax_combined = plot_spectrogram(combined_signal, fs, ax_combined, 'Combined Signal', window, nfft)
cb_combined = fig.colorbar(cax_combined, ax=ax_combined)
cb_combined.set_label('Intensity [dB]')

# Add figure title and settings
title = (f'Spectrograms of Measured Channels and PCA Combined Signal\n'
         f'Settings: Sampling Frequency = {fs} Hz, Window = "{window}", NFFT = {nfft}')
fig.suptitle(title, fontsize=12)

plt.tight_layout(rect=[0, 0, 1, 0.95])  # Adjust layout to make room for title
plt.show()

In [ ]:
plt.savefig("../test_local/images/4_Polarization__overview.png")

In [ ]:
# Prepare data scaler and scale data for training
scaler = MinMaxScaler()
data_normalized = scaler.fit_transform(combined_signal.reshape(-1, 1)).flatten()

In [ ]:
# Input data setup. The parameter and ny specify how many historic values from an input vector are be used
ny = 30
prediction_horizon = 1
N = data_normalized.shape[0]

X = np.ones((N, 1 + ny))  # On the first position we will keep bias
y = np.zeros((N,))

# Generate X, y dynamically based on 'ny'
for k in range(ny, N - prediction_horizon):
    X[k, 1:] = data_normalized[k - ny : k][::1]
    y[k] = data_normalized[k + prediction_horizon - 1]

X = X[ny:-prediction_horizon, :]
y = y[ny:-prediction_horizon]

In [ ]:
# Train Neural Unit
mu = 0.001  # Learning rate
epochs = 5

# Initialize parameters
N = X.shape[0]
print(f"The total amount of samples: {N}.")
nw = X.shape[1]  # Get weights for neuron
r = 1  # Set the order of polynom
if r > 1:
    nw = math.ceil(math.factorial(nw + r - 1) / math.factorial(r) / math.factorial(nw - 1))
w = np.random.randn(nw) / nw
print(f"Total amount of trainable weights {nw}.")

# Holders for adaptation values
e = np.zeros(N)
yn = np.zeros(N)
wall = np.zeros((N, nw))

# Training Quadratic Neural Unit
for epoch in range(epochs):
    for k in range(N):
        # Prepare polynomial input vector
        if r > 1:
            outer_product = np.outer(X[k], X[k])
            upper_triangle = np.triu_indices_from(outer_product)
            x = outer_product[upper_triangle]
        else:
            x = X[k, :]

        yn[k] = np.dot(w, x)
        e[k] = y[k] - yn[k]

        dyndw = x  # Gradient

        # Update weights
        # Uncomment the desired weight update rule

        # Pure Gradient Descent (GD)
        # dw = mu * e[k] * dyndw

        # GD with normalization
        dw = mu / (sum(x * x)) * e[k] * dyndw  # x0=1 => division by zero is avoided

        # GD with normalization and momentum
        # dw = 0.2 * mu / (sum(x * x)) * e[k] * dyndw + 0.8 * dw

        w = w + dw  # Update weights
        wall[k, :] = w  # Store weights for analysis

    epoch_error = np.mean(e**2)
    print(f"Epoch {epoch+1} Mean Squared Error: {epoch_error}")

In [ ]:
# Visualize model prediction
error = round(epoch_error, 5)
time_domain = np.arange(0, data.shape[0]) / fs
slice_start = 290
slice_end = 320


plt.figure(figsize=(8, 6))
# Plot observed target and neural output
ax1 = plt.subplot(411)
plt.title("Polarization changes compressed by PCA with LNU observer")
plt.plot(time_domain[ny:-prediction_horizon], y, "orange", label=r"$y ... observed \ targets$")
plt.plot(time_domain[ny:-prediction_horizon], yn, "g", label=r"$y_n ... neural \ outputs$")
plt.ylabel("[/]")
plt.legend(loc="upper left")
plt.xlim(time_domain[0], time_domain[-1])
plt.grid()

# Plot detain of signal
ax2 = plt.subplot(412)
plt.title(f"Detail of compressed polarization changes for window [{slice_start} - {slice_end}] seconds")
sample_range = np.arange(slice_start * fs, slice_end * fs) / fs
plt.plot(time_domain[ny:-prediction_horizon], y, "orange", label=r"$y ... observed \ targets$")
plt.plot(time_domain[ny + prediction_horizon :], yn, "g", label=r"$y_n ... neural \ outputs$")
plt.ylabel("[/]")
plt.xlim(sample_range[0], sample_range[-1])
plt.grid()

# Plot error between observed targets and neural outputs
ax3 = plt.subplot(413, sharex=ax1)
plt.plot(time_domain[ny + prediction_horizon :], e, "r", label=r"$e=y-y_n$")
plt.title(f"Error (MSE: {error})")
plt.ylabel("[/]")
plt.grid()

# Plot weights adaptation
ax4 = plt.subplot(414, sharex=ax1)
plt.plot(time_domain[ny + prediction_horizon :], wall[:, :10]), plt.ylabel(r"$\mathbf{w}(k)$")
plt.title(r"$\mathbf{w}(k)=\mathbf{w}(k-1)+\Delta \mathbf{w}(k)$")
plt.grid()
plt.xlabel(r"$Time$  [s]")

plt.tight_layout()
plt.show()


In [ ]:
plt.savefig("../test_local/images/4_AIXLEX_LNU.svg")

## AISLE
In this part we will demonstrate usage of Learning Entropy on anomaly detection. As explained in [[1](https://www.researchgate.net/publication/257200647_Learning_Entropy_Multiscale_Measure_for_Incremental_Learning)] and further refined in [[2](https://www.researchgate.net/publication/331041954_Learning_Entropy_as_a_Learning-Based_Information_Concept)], Learning Entropy detects an anomaly by analyzing the weight updates of an neuron. The basic concept expects that once model is trained, the changes to weight updates should remain relatively similar over observed window of weights unless there is some novelty in data, which should be detected by Learning Entropy, as model needs more _energy_ to achieve new state.

### Execution time comparisson
For comparison of pythonic approach and JAX accelerated execution we provide naive time measurement approach within this notebook. Please do note, that once the code has been executed, next unchanged executions are usually cached and therefore invalid for any comparisons. As a fair comparison serves the window implementation, which is too big to remain cached.

#### Disclaimer
Measuring execution times within Jupyter Notebook always brings certain amount of error and is supposed to be taken as indicative and not absolute. Single AISLE evaluation places JAX accelerated LE to an disadvantege, as on its initial run the code has to be compiled. Repeated runs are then already accelerated. If evaluated data are too short or weight count is low, the compilation time of JAX takes usually longer than native implementation.


#### Sources
[1] I. Bukovsky, “Learning Entropy: Multiscale Measure for Incremental Learning,” Entropy, vol. 15, no. 10, pp. 4159–4187, Sep. 2013, doi: 10.3390/e15104159.

[2] I. Bukovsky, W. Kinsner, and N. Homma, “Learning Entropy as a Learning-Based Information Concept,” Entropy, vol. 21, no. 2, p. 166, Feb. 2019, doi: 10.3390/e21020166.

### AISLE - Single value retrieval

In [ ]:
# Single AISLE value retrieval
oles = (1, 2)
alphas = (10, 15)

# Python single Learning Entropy evaluation
py_aisle_time = time.time()
py_aisle_value = py_aisle(wall, alphas=alphas, oles=oles)
py_aisle_time = time.time() - py_aisle_time

# JAX accelerated single Learning Entropy evaluation
jax_aisle_time = time.time()
jax_aisle_value = jax_aisle(wall, alphas=alphas, oles=oles)
jax_aisle_time = time.time() - jax_aisle_time

msg = (
    f"Single Learning Entropy evaluation with pythonic approach took: {py_aisle_time:.6f} seconds.\n",
    f"Single Learning Entropy evaluation with JAX aproach took: {jax_aisle_time:.6f} seconds.\n\n",
    "Single init evaluation with JAX is in general slower, as on its first run it has to compile parts of",
    " the code.\nRepeated usage is recommended.\n",
    "If reporting evaluation time is 0.000.., cache has been used and invalid time results are shown",
)
msg = "".join(text for text in msg)
print(msg)


### AISLE - batch processing (NumPy)

In [ ]:
# AISLE setup
oles = (3, 4)  # Order of LE
alphas = (6, 7)  # Sensitivity of evaluation
window = int(150 * 4.5)  # Window evaluated for each AISLE
threshold = 0.5  # Visual threshold (red dashed line)

# Sum, normalize and shift AISLE to correspond to original data
aisle_values = np.zeros(data.shape[0])
py_aisle_time = time.time()
py_aisle_values = py_aisle_window(window, wall, alphas, oles)
py_aisle_time = time.time() - py_aisle_time
aisle_values[ny:-prediction_horizon] = np.sum(py_aisle_values, axis=1) / len(oles)


In [ ]:
# Sort all indices in descending order
all_sorted_indices = np.argsort(aisle_values)[::-1]  # Sort all indices in descending order

# Define the range within which indices are considered too close
proximity_range = 2

# Filter indices that are too close, preferring the one with the higher value
filtered_indices = []
for index in all_sorted_indices:
    if aisle_values[index] < 0.1:
        break
    if not any(abs(index - other_index) <= proximity_range for other_index in filtered_indices):
        filtered_indices.append(index)

# Now select the top N indices after filtering
N = 5  # Number of top indices to select
final_indices = filtered_indices[:N]

# Construct the plot title with the top three significant values
top_values_str = "times:" + ", ".join([f" {idx/fs:.2f}" for idx in final_indices])
plot_title = (
    f"py_AISLE weight evaluation of ECG signal in {py_aisle_time:.2f} sec.\n",
    f"OLEs: {oles}, Alphas: {alphas}, window: {window}.\n",
    f"Top 5 significant detections at {top_values_str} sec.\n",
    "Area with train movement is marked with light green.",
)
plot_title = "".join(text for text in plot_title)

start = 21450
train_dt = start + 600*150
roi_ranges = (time_domain[start], time_domain[train_dt])
# roi_ranges = (start, train_dt)

# Plotting
fig = plt.figure(figsize=(8, 3))
fig.suptitle(plot_title)
plt.plot(time_domain, aisle_values, label="AISLE")
plt.hlines([0.5], 0, aisle_values.shape[0] // fs, colors="red", linestyles="dashed")
plt.ylabel(r"$[/]$")
plt.xlabel(r"$Time$  [s]")

# Fill specified ROI ranges with green background
plt.fill_between(
    x=np.arange(roi_ranges[0] + ny / fs, roi_ranges[1] + 1), y1=-0.05, y2=1.05, color="green", alpha=0.2
)

# Places crosses on top indices
for index in final_indices:
    if index < len(aisle_values):  # Check to avoid indexing errors
        plt.scatter(index / fs, aisle_values[index], color="red", marker="x", s=50)

plt.ylim((-0.05, 1.05))
plt.tight_layout()
plt.show()

In [ ]:
plt.savefig("../test_local/images/4_pyAISLE.svg")

### AISLE batch processing (JAX)

In [ ]:
# AISLEX setup
oles = (3, 4)  # Order of LE
alphas = (6, 7)  # Sensitivity of evaluation
window = int(150 * 4.5)  # Window evaluated for each AISLE
threshold = 0.5  # Visual threshold (red dashed line)
window_chunks = 35000

# Sum, normalize and shift AISLE to correspond to original data
aisle_values = np.zeros(data.shape[0])
jax_aisle_time = time.time()
jax_aisle_values = jax_aisle_window(window, wall, alphas, oles, window_chunks)
jax_aisle_time = time.time() - jax_aisle_time
aisle_values[ny + prediction_horizon :] = np.sum(jax_aisle_values, axis=1) / len(oles)

In [ ]:
# Sort all indices in descending order
all_sorted_indices = np.argsort(aisle_values)[::-1]  # Sort all indices in descending order

# Define the range within which indices are considered too close
proximity_range = 2

# Filter indices that are too close, preferring the one with the higher value
filtered_indices = []
for index in all_sorted_indices:
    if aisle_values[index] < 0.1:
        break
    if not any(abs(index - other_index) <= proximity_range for other_index in filtered_indices):
        filtered_indices.append(index)

# Now select the top N indices after filtering
N = 5  # Number of top indices to select
final_indices = filtered_indices[:N]

# Construct the plot title with the top three significant values
top_values_str = "times:" + ", ".join([f" {idx/fs:.2f}" for idx in final_indices])
plot_title = (
    f"jax_AISLE weight evaluation of ECG signal in {jax_aisle_time:.2f} sec.\n",
    f"OLEs: {oles}, Alphas: {alphas}, window: {window}.\n",
    f"Top 5 significant detections at {top_values_str} sec.\n",
    "Area with train movement is marked with light green.",
)
plot_title = "".join(text for text in plot_title)

start = 21450
train_dt = start + 600*150
roi_ranges = (time_domain[start], time_domain[train_dt])
# roi_ranges = (start, train_dt)

# Plotting
fig = plt.figure(figsize=(8, 3))
fig.suptitle(plot_title)
plt.plot(time_domain, aisle_values, label="AISLEX")
plt.hlines([0.5], 0, aisle_values.shape[0] // fs, colors="red", linestyles="dashed")
plt.ylabel(r"$[/]$")
plt.xlabel(r"$Time$  [s]")

# Fill specified ROI ranges with green background
plt.fill_between(
    x=np.arange(roi_ranges[0] + ny / fs, roi_ranges[1] + 1), y1=-0.05, y2=1.05, color="green", alpha=0.2
)

# Places crosses on top indices
for index in final_indices:
    if index < len(aisle_values):  # Check to avoid indexing errors
        plt.scatter(index / fs, aisle_values[index], color="red", marker="x", s=50)

plt.ylim((-0.05, 1.05))
plt.tight_layout()
plt.show()

In [ ]:
plt.savefig("../test_local/images/4_jaxAISLE.svg")

In [ ]:
# AISLE with pythonic approach
%timeit -n100 -r25 py_aisle(wall, alphas=alphas, oles=oles)

In [ ]:
# AISLE with JAX accelerated approach
%timeit -n100 -r25 jax_aisle(wall, alphas=alphas, oles=oles)

In [ ]:
# AISLE window evaluation with pythonic approach
%timeit -n5 -r10 py_aisle_window(window, wall, alphas, oles)

In [ ]:
# AISLE window evaluation with JAX accelerated approach
%timeit -n5 -r10 jax_aisle_window(window, wall, alphas, oles)